In [1]:
import sqlite3

In [2]:
#create database
db = sqlite3.connect("Gilbert_Wedge.db") #':memory:') # Make this a directory + file if you want to store the results.
cur = db.cursor()
db.close()
#init_db(cur)

In [2]:
#imports for the code
from google.cloud import bigquery
from google.oauth2 import service_account
import os

In [3]:
#establish pathways
service_path = 'C:\\Users\Jon\Desktop\Grad school\Applied Data Analytics\Wedge\ADA-Wedge\Wedge Zip Big\\'
service_file = 'gilbert-ada-wedgeproject-75116ed694f5.json' # change this to your authentication information  
gbq_proj_id = 'gilbert-ada-wedgeproject' # change this to your poroject. 
gbq_dataset_id = 'gilbert_wedge'

private_key =service_path + service_file

In [4]:
#use credentials to establish GBQ connection
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [6]:
#query for first table

query = ('''
SELECT
(EXTRACT(date FROM datetime)) AS date,
(EXTRACT(hour FROM datetime)) AS hour,
  SUM(total) AS spend,
  COUNT(DISTINCT(DATE(datetime) || register_no || emp_no || trans_no)) as Transactions,
  sum(CASE WHEN (trans_status = 'V' or trans_status = 'R') THEN -1 ELSE 1 END) as Items
FROM `gilbert-ada-wedgeproject.gilbert_wedge.gilbert_wedge_full`
WHERE
  card_no != 3
  AND department != 0
  AND department != 15
  AND trans_status != 'M'
  AND trans_status != 'C'
  AND trans_status != 'J'
  AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status = 'V'
    OR trans_status = 'R')
GROUP BY date, hour
ORDER BY date, hour
''')

query_job = client.query(
    query,
    location = 'US',
)

table_one = client.query(query)



with open('table1.txt', 'w')as output_file:
    for line in table_one:
        row = [str(item) for item in line]
        output_file.write(','.join(row)+ '\n')

In [14]:
#create table1 in db
input_file = 'table1.txt'
db = sqlite3.connect("Gilbert_Wedge.db")
cur = db.cursor()

cur.execute('''DROP TABLE IF EXISTS Table1''')
cur.execute('''CREATE TABLE Table1 (
    date TIMESTAMP,
    hour TIMESTAMP,
    sales REAL,
    transactions INTEGER,
    items TEXT)''')



In [15]:
#insert query 1 data into table1
with open(input_file, 'r', encoding = 'Latin-1') as infile:
    for idx,line in enumerate(infile.readlines()):
        line = line.strip().split(',')
        cur.execute('''
        INSERT INTO Table1 (date, hour, sales, transactions, items)
        VALUES (?,?,?,?,?)''', line)
db.commit()

In [17]:
#query for second table

query = ('''
SELECT
card_no,
(EXTRACT(year FROM datetime)) AS year,
(EXTRACT(month FROM datetime)) AS month,
  SUM(total) AS spend,
  COUNT(DISTINCT(DATE(datetime) || register_no || emp_no || trans_no)) as Transactions,
  sum(CASE WHEN (trans_status = 'V' or trans_status = 'R') THEN -1 ELSE 1 END) as Items,

FROM `gilbert-ada-wedgeproject.gilbert_wedge.gilbert_wedge_full`
WHERE
  card_no != 3
  AND department != 0
  AND department != 15
  AND trans_status != 'M'
  AND trans_status != 'C'
  AND trans_status != 'J'
  AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status = 'V'
    OR trans_status = 'R')
GROUP BY card_no, year, month
ORDER BY year, month
''')

query_job = client.query(
    query,
    location = 'US',
)

table_two = client.query(query)



with open('table2.txt', 'w')as output_file:
    for line in table_two:
        row = [str(item) for item in line]
        output_file.write(','.join(row)+ '\n')

In [18]:
#create table2 in db
input_file = 'table2.txt'
db = sqlite3.connect("Gilbert_Wedge.db")
cur = db.cursor()

cur.execute('''DROP TABLE IF EXISTS Table2''')
cur.execute('''CREATE TABLE Table2 (
    card_no INTEGER,
    year TIMESTAMP,
    month TIMESTAMP,
    sales REAL,
    transactions INTEGER,
    items TEXT)''')

In [19]:
#insert query 2 data into table2
with open(input_file, 'r', encoding = 'Latin-1') as infile:
    for idx,line in enumerate(infile.readlines()):
        line = line.strip().split(',')
        cur.execute('''
        INSERT INTO Table2 (card_no, year, month, sales, transactions, items)
        VALUES (?,?,?,?,?,?)''', line)
db.commit()

In [20]:
#query for third table

query = ('''
SELECT
upc,
description,
a.department,
b.department_name,
  
(EXTRACT(year FROM datetime)) AS year,
(EXTRACT(month FROM datetime)) AS month,
  SUM(total) AS spend,
  COUNT(DISTINCT(DATE(datetime) || register_no || emp_no || trans_no)) as Transactions,
  sum(CASE WHEN (trans_status = 'V' or trans_status = 'R') THEN -1 ELSE 1 END) as Items,

FROM `gilbert-ada-wedgeproject.gilbert_wedge.gilbert_wedge_full`a
LEFT JOIN `gilbert-ada-wedgeproject.gilbert_wedge.dept_name`b
ON a.department = b.department
WHERE
  card_no != 3
  AND a.department != 0
  AND a.department != 15
  AND trans_status != 'M'
  AND trans_status != 'C'
  AND trans_status != 'J'
  AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status = 'V'
    OR trans_status = 'R')
GROUP BY upc, description, department, b.department_name,year, month
ORDER BY year, month
''')

query_job = client.query(
    query,
    location = 'US',
)

table_two = client.query(query)



with open('table3.txt', 'w')as output_file:
    for line in table_two:
        row = [str(item) for item in line]
        output_file.write(','.join(row)+ '\n')

In [23]:
#create table3 in db
input_file = 'table3.txt'
db = sqlite3.connect("Gilbert_Wedge.db")
cur = db.cursor()

cur.execute('''DROP TABLE IF EXISTS Table3''')
cur.execute('''CREATE TABLE Table3 (
    upc STRING,
    description STRING,
    department FLOAT,
    department_name STRING,
    year TIMESTAMP,
    month TIMESTAMP,
    sales REAL,
    transactions INTEGER,
    items TEXT)''')

In [24]:
#insert query 3 data into table3
with open(input_file, 'r', encoding = 'Latin-1') as infile:
    for idx,line in enumerate(infile.readlines()):
        line = line.strip().split(',')
        cur.execute('''
        INSERT INTO Table3 (upc, description, department, department_name, year, month, sales, transactions, items)
        VALUES (?,?,?,?,?,?,?,?,?)''', line)
db.commit()